#import

In [ ]:
!python -c 'import keras; print("Keras version "+ keras.__version__)'

In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as pdr #data 받아오는 api 라이브러리
from pandas import json_normalize
import numpy as np
import plotly as py
import plotly.graph_objs as go
import requests
from datetime import datetime
import traceback
import os
import time
from datetime import timedelta

import tensorflow as tf
from keras import metrics
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from sklearn.preprocessing import MinMaxScaler

plt.style.use('bmh')
print("Tensorflow version " + tf.__version__)

In [ ]:
!pwd

In [ ]:
#사용할 GPU 설정 재설정하려면 프로세스 재시작해야함
os.environ['CUDA_VISIBLE_DEVICES']='0'#tf.debugging.set_log_device_placement(True) #어떤 장치가 할당되었는지 확인 용도인데 에러뜸

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # 텐서플로가 첫 번째 GPU만 사용하도록 제한
  try:
    tf.config.experimental.set_visible_devices(gpus[:], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 합니다
    print(e)
gpus

# Request URL API

In [ ]:
# #BTC
# start_date = datetime(2014,7,18)
# start = int(start_date.timestamp()) #꼭 tuple이었던 변수를 datetime으로 넣을 때는 *을 붙여줘야함
# period = 86400
# #300, 900, 1800, 7200, 14400, and 86400.
# #5분, 15분, 30분, 2시간, 4시간, 1일
# ret = requests.get('https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start={}&end=9999999999&period={}'.format(start, period))

In [ ]:
stock_url = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] #[0] 없으면 DataFrame이 아니라 list
stock_url.종목코드 = stock_url.종목코드.map('{:06d}'.format)
stock_url = stock_url[['회사명', '종목코드']] #필요한 두가지만
stock_url = stock_url.rename(columns={'회사명':'name', '종목코드':'code'})

In [ ]:
item_name = 'SK하이닉스'
code = stock_url.query("name=='{}'".format(item_name))['code'].to_string(index=False).strip() #strip 앞뒤공백제거
dr_code = code + '.KS' #DataReader용, 코스닥은 KQ
folder_name = code +' '+ item_name
print(item_name, dr_code)

# DataReader crawling

In [ ]:
#URL 없이 상당히 쉬운 DataReader를 이용한 크롤링
start = (2012, 1, 1)
start_date = datetime(*start)
end = datetime.today()
df = pdr.DataReader(dr_code, 'yahoo', start_date, end) 

In [ ]:
scaler = MinMaxScaler()
real_price = df['Close'].tolist()
price = scaler.fit_transform(df[['Close']]).reshape(-1).tolist()
print('오늘은',datetime.today(), real_price[-1], len(real_price), '개')

In [ ]:
def make_DataSet():
    #make Dataset
    x = []  
    y = []  

    for i in range(len(price) - window_size):  
        x.append([price[i+j] for j in range(window_size)])
        y.append(price[window_size + i])

    x = np.asarray(x)
    y = np.asarray(y)

    x_test = x[:train_test_split, :]
    y_test = y[:train_test_split]

    x_train = x[train_test_split:, :]
    y_train = y[train_test_split:]

    xtrain = x_train.reshape((x_train.shape[0], x_train.shape[1], 1)) 
    xtest =  np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1)) 

    return xtrain, xtest, y_train, y_test


In [ ]:
def makeplot(TITLE):
    split_pt = train_test_split + window_size
    
    trace = go.Scatter(x=np.arange(0,len(price),1), y=price, 
    mode = 'lines', name='actual')
    trace2 = go.Scatter(x=np.arange(window_size,split_pt,1), y=test_pred.reshape(len(test_pred)), 
    mode = 'lines', name='train')
    trace3 = go.Scatter(x=np.arange(split_pt,split_pt + len(train_pred),1), y=train_pred.reshape(len(train_pred)),  
    mode = 'lines', name='prediction')

    data = [trace, trace2, trace3]
    layout = go.Layout(title=TITLE)
    fig = go.Figure(data=data, layout=layout)
    #py.offline.iplot(fig) #그래프 출력 
    py.offline.plot(fig, show_link = True, filename = TITLE + '.html') #다른이름으로 지정
    #os.rename('temp-plot.html', rename)
    #files.download(filename=Title +'.html') #다운로드


# Args

In [ ]:
#history Metrics
import keras
class CustomHistory(keras.callbacks.Callback):
  def init(self):
    self.losses = []
    self.accs = []
  def on_epoch_end(self, batch, logs={}):
    self.losses.append(logs.get('loss'))
    self.accs.append(logs.get('acc'))   

In [ ]:
from keras import optimizers
#args
n_Memcell = 128
window_size = 60 
train_test_split = int((len(price)-window_size)*0.3)
predict_day = 10

#return_sequences = 여러 층일때, 각 시퀀스에서 출력할지 말지 결정. many to many 일때
b_state = True
n_batch = 1
n_epoch = 1000 
n_drop = 0.2
learning_rate = 0.001 #default
ac_func = 'relu'
loss_func = 'mse'
optim_func = optimizers.Adam(lr=learning_rate)
#decay_rate = 0.001 #신경망에서는 수학적으로 동일한 L2 놈을 대신사용한다.
#optim_func = optimizers.Adadelta(learning_rate = learning_rate) #0.1, 0.001 둘다 과소적합

args_str = '['+str(datetime.date(start_date)) + ' ~ ' + str(datetime.date(datetime.today())) + '] ' + str(n_Memcell) + 'cell '+ str(window_size) +'win '+ str(n_batch) +'bat ' + str(n_epoch) + 'ep '+ str(n_drop) + 'drop ' + str(b_state) + ' ' + str(learning_rate) + 'lr'
Title = item_name + ': ' + args_str
Title

# Iterative Training (in GPU server)

In [ ]:
cd /home/oslab/csj/notebooks/Models/New Models

In [ ]:
for i in range(0, 200, int(200/8)):
    print('kospi_data = stock_data[%d:%d]' %(i,i+ int(200 / 8)))


In [ ]:
cd /home/oslab/csj/notebooks/Models/New Models

In [ ]:
#KOSPI200 Trainning
#load csv
stock_data = pd.read_csv('KOSPI200.csv')
stock_data.종목코드 = stock_data.종목코드.map('{:06d}'.format) 
stock_data = stock_data.rename(columns={'회사명':'name', '종목코드':'code'})

#종목 전부다 학습하기
kospi_data = stock_data[0:25]
for idx,it in kospi_data.iterrows():
    Training_start = time.time()
    #folder_name = it[0] + ' ' + it[1]
    item_name = it[1]
    dr_code = it[0] + '.KS'
    print(item_name, dr_code)
    
    start = (2012, 1, 1)
    start_date = datetime(*start)
    end = datetime.today()
    df = pdr.DataReader(dr_code, 'yahoo', start_date, end)
    
    scaler = MinMaxScaler()
    real_price = df['Close'].tolist()
    price = scaler.fit_transform(df[['Close']]).reshape(-1).tolist()
    print('오늘은',datetime.today(), real_price[-1], len(real_price), '개')
    
    xtrain, xtest, y_train, y_test = make_DataSet(price, window_size) 
    
     #model define
    model = Sequential()
    model.add(LSTM(n_Memcell, batch_input_shape =(n_batch, window_size, 1), stateful=b_state))
    model.add(Dropout(n_drop))
    model.add(Activation(ac_func))
    model.add(Dense(1))
    model.compile(loss=loss_func, optimizer=optim_func, metrics=['mae', 'acc'])

    #training
    report = CustomHistory()
    report.init()
    early_stopping = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=10, mode='auto') 

    for i in range(n_epoch):
        print('[%d] ' %i)
        model.fit(xtrain, y_train, epochs=1, batch_size=n_batch, callbacks=[report, early_stopping],verbose=2)
        model.reset_states()
    
    sec = time.time() - Training_start
    times = str(timedelta(seconds=sec)).split(".")
    print('Training Time: %s' %times[0])

    #save the model
    from keras.models import load_model
    args_str = '['+str(datetime.date(start_date)) + ' ~ ' + str(datetime.date(datetime.today())) + '] ' + str(n_Memcell) + 'cell '+ str(window_size) +'win '+ str(n_batch) +'bat ' + str(n_epoch) + 'ep '+ str(n_drop) + 'drop ' + str(b_state) + ' ' + str(learning_rate) + 'lr'
    Title = item_name + ': ' + args_str
    file = Title + '.h5'
    model.save(file) 

    #move folder
    import shutil
    src = '/home/oslab/csj/notebooks/Models/New Models/'
    des = '/home/oslab/csj/notebooks/Models/' + folder_name + '/'
    shutil.move(src + file, des + file)
    
    #predict
    train_pred = model.predict(xtrain,batch_size=n_batch)
    model.reset_states()
    test_pred = model.predict(xtest,batch_size=n_batch)
    model.reset_states()
    after_pred = model.predict(xtest,batch_size=n_batch)
    model.reset_states()

    #evaluate
    trainScore = model.evaluate(xtrain, y_train, batch_size=n_batch, verbose=0)
    testScore = model.evaluate(xtest, y_test, batch_size=n_batch, verbose=0)
    print("(%.5f, %.5f)" %(trainScore[1], testScore[1]))
    
    makeplot(Title)

from numba import cuda
cuda.select_device(0)
cuda.close()    

# Best Args 찾기

In [ ]:
#cd /home/oslab/csj/notebooks/Models/$folder_name

In [ ]:
# #Best Args 찾기

# xaxes = []
# train_maes = []
# test_maes = []

# for w in range(5, 125, 5):
#     #Expriment args
#     Training_start = time.time()
#     window_size = w
#     #   if n_epoch > 2000: #하한
#     #     n_epoch = n_epoch - 100

#     #make Dataset
#     xtrain, xtest, y_train, y_test = make_DataSet(price, window_size)

#     #model define
#     model = Sequential()
#     model.add(LSTM(n_Memcell, batch_input_shape =(n_batch, window_size, 1), stateful=b_state))
#     model.add(Dropout(n_drop))
#     model.add(Activation(ac_func))
#     model.add(Dense(1))
#     model.compile(loss=loss_func, optimizer=optim_func, metrics=['mae', 'acc'])

#     #training
#     report = CustomHistory()
#     report.init()
#     early_stopping = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=10, mode='auto') 


#     for i in range(n_epoch):
#         print('[%d] ' %i)
#         model.fit(xtrain, y_train, epochs=1, batch_size=n_batch, callbacks=[report, early_stopping],verbose=2)
#         model.reset_states()

#     sec = time.time() - Training_start
#     times = str(timedelta(seconds=sec)).split(".")
#     print('Training Time: %s' %times[0])

#     #save the model
#     from keras.models import load_model
#     args_str = '['+str(datetime.date(start_date)) + ' ~ ' + str(datetime.date(datetime.today())) + '] ' + str(n_Memcell) + 'cell '+ str(window_size) +'win '+ str(n_batch) +'bat ' + str(n_epoch) + 'ep '+ str(n_drop) + 'drop ' + str(b_state) + ' ' + str(learning_rate) + 'lr'
#     Title = item_name + ': ' + args_str
#     file = Title + '.h5'
#     model.save(file)

#     #move folder
#     import shutil
#     src = '/home/oslab/csj/notebooks/Models/New Models/'
#     des = '/home/oslab/csj/notebooks/Models/' + folder_name + '/'
#     shutil.move(src + file, des + file)

#     #evaluate
#     trainScore = model.evaluate(xtrain, y_train, batch_size=n_batch, verbose=0)
#     testScore = model.evaluate(xtest, y_test, batch_size=n_batch, verbose=0)
#     #print('%d epoch' %n_epoch)  #Expriment args
#     print('%d win' %window_size) #Expriment args
#     print("(%.5f, %.5f)" %(trainScore[1], testScore[1]))
#     print(np.average(report.losses))

# Model Define

In [ ]:
#Model Define-GPU 할당 시작

model = Sequential()
model.add(LSTM(n_Memcell, batch_input_shape =(n_batch, window_size, 1), stateful=b_state))
model.add(Dropout(n_drop))
model.add(Dense(1))

model.compile(loss=loss_func, optimizer=optim_func, metrics=['mae', 'acc'])
model.summary()

# Triaining

In [ ]:
xtrain, xtest, y_train, y_test = make_DataSet(price, window_size)

report = CustomHistory()
report.init()
early_stopping = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=10, mode='auto') #loss가 증가하면 훈련 그만하기 loss는 증가안함 acc가 감소하면으로 바꿔주기 
Training_start = time.time()
for i in range(n_epoch):
    print('[%d] ' %i)
    model.fit(xtrain, y_train, epochs=1, batch_size=n_batch, callbacks=[report, early_stopping],verbose=2)
    model.reset_states()

sec = time.time() - Training_start
times = str(timedelta(seconds=sec)).split(".")
print('Training Time: %s' %times[0])

# Save Model

In [ ]:
from keras.models import load_model
model.save(Title + '.h5')

# Evaluate

In [ ]:
trainScore = model.evaluate(xtrain, y_train, batch_size=n_batch, verbose=0)
testScore = model.evaluate(xtest, y_test, batch_size=n_batch, verbose=0)
print("mae: (%.5f, %.5f)" %(trainScore[1], testScore[1]))
#trainScore[1]

In [ ]:
print(trainScore)
testScore

In [ ]:
loss_ax = plt.subplot()
acc_ax = loss_ax.twinx()

loss_ax.plot(report.losses, 'y', label='train_loss')
acc_ax.plot(report.accs, 'b', label='train_acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()


# Predict

In [ ]:
#train, test prediction
train_pred = model.predict(xtrain,batch_size=n_batch) # x[0~window_size-1] 만큼의 특징벡터를 학습해서 학습데이터 수 
#model.reset_states()
test_pred = model.predict(xtest,batch_size=n_batch)
#model.reset_states()
real_test_pred = scaler.inverse_transform(test_pred)

In [ ]:
tom_real = xtrain[-1]
tom_real = np.vstack((tom_real[1:], price[-1])) #test_pred[-1] 대신에 price[-1]
tom_real.shape

tom_pred = xtest[-1]
tom_pred = np.vstack((tom_pred[1:], test_pred[-1])) #마지막 데이터에 예측값
tom_pred.shape

In [ ]:
#학습된 모델로 예측하기
seq_in = tom_real #오늘꺼 예측자를 마지막 데이터로 추가해준 Input 샘플 or real 데이터 추가
#first_in = xtest[-1]  #test 샘플 마지막. 여기에다가 다음날 pred를 추가해줄거임
after_pred = np.zeros((predict_day,1))

for i in range(predict_day):
  sample_in = seq_in.reshape(1,window_size,1)
  after = model.predict(sample_in, batch_size=n_batch)
  model.reset_states()
  seq_in = np.vstack((seq_in[1:], after))
  after_pred[i] = after
  
#model.reset_states()
real_after_pred = scaler.inverse_transform(after_pred)

In [ ]:
#학습결과 그래프
split_pt = train_test_split + window_size
plt.figure(figsize=(20,10))
plt.plot(price)

plt.plot(np.arange(0, len(price),1), price, color='b')
plt.plot(np.arange(window_size, split_pt, 1), test_pred, color='g')
plt.plot(np.arange(split_pt, split_pt + len(train_pred), 1), train_pred, color='r')
plt.plot(np.arange(len(price), len(price) + len(after_pred), 1), after_pred, color='y')

print(
    '현재 종가: ', price[-1], #x는 마지막값 누락
    '예측 종가: ', test_pred[-1],
    '오 차 율: ',  (price[-1] - test_pred[-1]) / price[-1], sep='\n')

In [ ]:
makeplot(Title)

In [ ]:
print('현재 종가: ', real_price[-1], #df, series는 -1 인덱싱 안됨
      '\n다음 예측가격: \n', real_after_pred)

In [ ]:
plt.plot(np.arange(0,len(after_pred), 1), after_pred, color='red')

In [ ]:
#작업후 GPU메모리 반납
from numba import cuda
cuda.select_device(0)
cuda.close()